In [3]:
import os
import copy
import pickle
from pprint import pp
from rdkit import Chem
from rdkit.Chem import AllChem
from openfe.setup import Network
from openfe.setup import SmallMoleculeComponent
from openfe.setup.ligand_network_planning import generate_radial_network
from openfe.setup.lomap_mapper import LomapAtomMapper
from openfe.setup.methods.openmm.equil_rbfe_methods import (
    SystemSettings, TopologySettings, AlchemicalSettings,
    OpenMMEngineSettings, SamplerSettings, BarostatSettings,
    IntegratorSettings, SimulationSettings
)
from openfe.setup.methods.openmm.equil_rbfe_methods import RelativeLigandTransformSettings
from openfe.setup.methods.openmm.equil_rbfe_methods import RelativeLigandTransform
import openfe.setup.methods.openmm.protein_preparation as pp
from openfe.setup import ChemicalSystem
from openfe.setup import SolventComponent, ProteinComponent
from openfe.setup import SLURM
from openff.units import unit
from openfe.utils.atommapping_network_plotting import plot_atommapping_network

In [14]:
os.chdir('/Users/evianne/Downloads/openfe-main/simulation/')
params = SLURM.read_params('params.txt')
os.chdir(params['Submit_directory'])
#extract ligands
ligand_sdf = Chem.SDMolSupplier(params['ligand_file'], removeHs=False)

ligand_mols = [SmallMoleculeComponent(sdf) for sdf in ligand_sdf]

# Create a network - reference molecule is ligand 0
ligand_network = generate_radial_network(ligands=ligand_mols[1:],
                                       central_ligand=ligand_mols[0],
                                       mappers=[LomapAtomMapper(threed=True),])

with open("network_store.graphml", "w") as writer:
    writer.write(ligand_network.to_graphml())

In [15]:
# Define the Protein and Solvent Components which we will be using
protein = ProteinComponent.from_pdbfile(params['PDB_file'], name=params['Protein_name'])
solvent = SolventComponent(positive_ion='Na', negative_ion='Cl',
                           neutralize=True, ion_concentration=0.15*unit.molar)


In [16]:
n=1
for edge in ligand_network.edges:
    os.chdir(params['Submit_directory'])
    #path = os.path.join(params['Submit_directory'], "/simulation/")
    #parent_dir = params['Submit_directory'] + "/simulation/"
    parent_dir = params['Submit_directory']
    path = os.path.join(parent_dir, "molecule"+str(n))
    os.mkdir(path)
    print("Directory '% s' created" % path)
    os.chdir(path)
    molA_to_molB = edge
    molA_complex = ChemicalSystem({'ligand': molA_to_molB.molA,
                                  'solvent': solvent,
                                  'protein': protein,})
    molA_solvent = ChemicalSystem({'ligand': molA_to_molB.molA,
                                  'solvent': solvent,})

    molB_complex = ChemicalSystem({'ligand': molA_to_molB.molB,
                                 'solvent': solvent,
                                 'protein': protein,})
    molB_solvent = ChemicalSystem({'ligand': molA_to_molB.molB,
                                 'solvent': solvent,})
    settings = SLURM.updated_params(params)
    complex_settings = RelativeLigandTransformSettings(system_settings=settings[0],topology_settings=settings[1],
                                            alchemical_settings=settings[2],sampler_settings=settings[3],
                                            barostat_settings=settings[4],integrator_settings=settings[5],
                                            simulation_settings=settings[6])

    solvent_settings = copy.deepcopy(complex_settings)

# Set the complex output file names
    complex_settings.simulation_settings.output_filename = 'complex_rbfe_.nc'
    complex_settings.simulation_settings.checkpoint_storage = 'complex_rbfe_checkpoint.nc'

# Set the solvent output file names
    solvent_settings.simulation_settings.output_filename = 'solvent_rbfe.nc'
    solvent_settings.simulation_settings.checkpoint_storage = 'solvent_rbfe_checkpoint.nc'

    print(complex_settings.simulation_settings)
    complex_transform = RelativeLigandTransform(
        stateA=molA_complex, stateB=molB_complex,
        ligandmapping=molA_to_molB,
        settings=complex_settings
    )
    
    solvent_transform = RelativeLigandTransform(
        stateA=molA_solvent, stateB=molB_solvent,
        ligandmapping=molA_to_molB,
        settings=solvent_settings
    )
    
#dry-runs
    complex_transform.run(dry=True, verbose=True)
    solvent_transform.run(dry=True, verbose=True)
    SLURM.save_object(complex_transform, "complex_transform.pkl")
    SLURM.save_object(solvent_transform, "solvent_transform.pkl")
    slurmsettings2=SLURM.SLURMSettings.parse_raw(params['SLURM_settings_job'])
    SLURM.write_job_script(slurmsettings2, filename="job_script.sh")
    n=n+1
    print(n)
    !rm complex_rbfe_checkpoint.nc solvent_rbfe_checkpoint.nc 
os.chdir(parent_dir)
slurmsettingsMaster = SLURM.SLURMSettingsM.parse_raw(params['SLURM_settings_master'])
SLURM.write_master_script(slurmsettingsMaster, filename="job_master_script.sh", nof_jobs=len(ligand_network.edges))

INFO:	creating hybrid system


Directory '/Users/evianne/Downloads/openfe-main/simulation/molecule1' created
equilibration_length=<Quantity(2.0, 'nanosecond')> production_length=<Quantity(5.0, 'nanosecond')> checkpoint_storage='complex_rbfe_checkpoint.nc' minimization_steps=10000 output_filename='complex_rbfe_.nc' output_indices='all' checkpoint_interval=<Quantity(50, 'timestep')>


INFO:	Requested to generate parameters for residue <Residue 164 (benzene) of chain 1>
INFO:	Generating a residue template for [H][c]1[c]([H])[c]([H])[c]([H])[c]([H])[c]1[H] using openff-2.0.0
  <AtomTypes>
    <Type name="[H][c]1[c]([H])[c]([H])[c]([H])[c]([H])[c]1[H]$C1#0" element="C" mass="12.01078" class="[H][c]1[c]([H])[c]([H])[c]([H])[c]([H])[c]1[H]$C1#0"/>
    <Type name="[H][c]1[c]([H])[c]([H])[c]([H])[c]([H])[c]1[H]$C2#1" element="C" mass="12.01078" class="[H][c]1[c]([H])[c]([H])[c]([H])[c]([H])[c]1[H]$C2#1"/>
    <Type name="[H][c]1[c]([H])[c]([H])[c]([H])[c]([H])[c]1[H]$C3#2" element="C" mass="12.01078" class="[H][c]1[c]([H])[c]([H])[c]([H])[c]([H])[c]1[H]$C3#2"/>
    <Type name="[H][c]1[c]([H])[c]([H])[c]([H])[c]([H])[c]1[H]$C4#3" element="C" mass="12.01078" class="[H][c]1[c]([H])[c]([H])[c]([H])[c]([H])[c]1[H]$C4#3"/>
    <Type name="[H][c]1[c]([H])[c]([H])[c]([H])[c]([H])[c]1[H]$C5#4" element="C" mass="12.01078" class="[H][c]1[c]([H])[c]([H])[c]([H])[c]([H])[c]1[H]$C5#4"/>

dict_keys([(0, 1, 2, 3), (0, 1, 2, 8), (0, 5, 4, 3), (0, 5, 4, 10), (1, 0, 5, 4), (1, 0, 5, 11), (1, 2, 3, 4), (1, 2, 3, 9), (2, 1, 0, 5), (2, 1, 0, 6), (2, 3, 4, 5), (2, 3, 4, 10), (3, 2, 1, 7), (3, 4, 5, 11), (4, 3, 2, 8), (4, 5, 0, 6), (5, 0, 1, 7), (5, 4, 3, 9), (6, 0, 1, 7), (6, 0, 5, 11), (7, 1, 2, 8), (8, 2, 3, 9), (9, 3, 4, 10), (10, 4, 5, 11), (1, 0, 2, 7), (1, 2, 7, 0), (1, 7, 0, 2), (5, 0, 4, 11), (5, 4, 11, 0), (5, 11, 0, 4), (0, 1, 5, 6), (0, 5, 6, 1), (0, 6, 1, 5), (2, 1, 3, 8), (2, 3, 8, 1), (2, 8, 1, 3), (3, 2, 4, 9), (3, 4, 9, 2), (3, 9, 2, 4), (4, 3, 5, 10), (4, 5, 10, 3), (4, 10, 3, 5)])
<Element Proper at 0x1723c9d00>
<Element Proper at 0x1723c9600>
<Element Proper at 0x1723c9780>
<Element Proper at 0x1723c9b80>
<Element Proper at 0x172290e80>
<Element Proper at 0x172290700>
<Element Proper at 0x172290c00>
<Element Proper at 0x172290dc0>
<Element Proper at 0x172290400>
<Element Proper at 0x172290100>
<Element Proper at 0x172290ec0>
<Element Proper at 0x172290b80>
<E

INFO:	Requested to generate parameters for residue <Residue 11856 (phenol) of chain 4>
INFO:	Generating a residue template for [H][O][c]1[c]([H])[c]([H])[c]([H])[c]([H])[c]1[H] using openff-2.0.0
  <AtomTypes>
    <Type name="[H][O][c]1[c]([H])[c]([H])[c]([H])[c]([H])[c]1[H]$C1#0" element="C" mass="12.01078" class="[H][O][c]1[c]([H])[c]([H])[c]([H])[c]([H])[c]1[H]$C1#0"/>
    <Type name="[H][O][c]1[c]([H])[c]([H])[c]([H])[c]([H])[c]1[H]$C2#1" element="C" mass="12.01078" class="[H][O][c]1[c]([H])[c]([H])[c]([H])[c]([H])[c]1[H]$C2#1"/>
    <Type name="[H][O][c]1[c]([H])[c]([H])[c]([H])[c]([H])[c]1[H]$C3#2" element="C" mass="12.01078" class="[H][O][c]1[c]([H])[c]([H])[c]([H])[c]([H])[c]1[H]$C3#2"/>
    <Type name="[H][O][c]1[c]([H])[c]([H])[c]([H])[c]([H])[c]1[H]$C4#3" element="C" mass="12.01078" class="[H][O][c]1[c]([H])[c]([H])[c]([H])[c]([H])[c]1[H]$C4#3"/>
    <Type name="[H][O][c]1[c]([H])[c]([H])[c]([H])[c]([H])[c]1[H]$C5#4" element="C" mass="12.01078" class="[H][O][c]1[c]([H])[c]([

...

INFO:	Requested to generate parameters for residue <Residue 736 (benzonitrile) of chain 3>
INFO:	Generating a residue template for [H][c]1[c]([H])[c]([H])[c]([C]#[N])[c]([H])[c]1[H] using openff-2.0.0
  <AtomTypes>
    <Type name="[H][c]1[c]([H])[c]([H])[c]([C]#[N])[c]([H])[c]1[H]$N1#0" element="N" mass="14.00672" class="[H][c]1[c]([H])[c]([H])[c]([C]#[N])[c]([H])[c]1[H]$N1#0"/>
    <Type name="[H][c]1[c]([H])[c]([H])[c]([C]#[N])[c]([H])[c]1[H]$C1#1" element="C" mass="12.01078" class="[H][c]1[c]([H])[c]([H])[c]([C]#[N])[c]([H])[c]1[H]$C1#1"/>
    <Type name="[H][c]1[c]([H])[c]([H])[c]([C]#[N])[c]([H])[c]1[H]$C2#2" element="C" mass="12.01078" class="[H][c]1[c]([H])[c]([H])[c]([C]#[N])[c]([H])[c]1[H]$C2#2"/>
    <Type name="[H][c]1[c]([H])[c]([H])[c]([C]#[N])[c]([H])[c]1[H]$C3#3" element="C" mass="12.01078" class="[H][c]1[c]([H])[c]([H])[c]([C]#[N])[c]([H])[c]1[H]$C3#3"/>
    <Type name="[H][c]1[c]([H])[c]([H])[c]([C]#[N])[c]([H])[c]1[H]$C4#4" element="C" mass="12.01078" class="[H][c]1[c

INFO:	creating hybrid system
INFO:	setting force field terms
INFO:	adding forces


dict_keys([(0, 1, 6, 5), (0, 1, 6, 7), (1, 6, 5, 4), (1, 6, 5, 11), (1, 6, 7, 2), (1, 6, 7, 12), (2, 3, 4, 5), (2, 3, 4, 10), (2, 7, 6, 5), (3, 2, 7, 6), (3, 2, 7, 12), (3, 4, 5, 6), (3, 4, 5, 11), (4, 3, 2, 7), (4, 3, 2, 8), (4, 5, 6, 7), (5, 4, 3, 9), (5, 6, 7, 12), (6, 5, 4, 10), (6, 7, 2, 8), (7, 2, 3, 9), (7, 6, 5, 11), (8, 2, 3, 9), (8, 2, 7, 12), (9, 3, 4, 10), (10, 4, 5, 11), (6, 1, 5, 7), (6, 5, 7, 1), (6, 7, 1, 5), (3, 2, 4, 9), (3, 4, 9, 2), (3, 9, 2, 4), (7, 2, 6, 12), (7, 6, 12, 2), (7, 12, 2, 6), (2, 3, 7, 8), (2, 7, 8, 3), (2, 8, 3, 7), (4, 3, 5, 10), (4, 5, 10, 3), (4, 10, 3, 5), (5, 4, 6, 11), (5, 6, 11, 4), (5, 11, 4, 6)])
<Element Proper at 0x17f350c80>
<Element Proper at 0x17f350f40>
<Element Proper at 0x17f3508c0>
<Element Proper at 0x17f350d80>
<Element Proper at 0x17f350dc0>
<Element Proper at 0x17f44a700>
<Element Proper at 0x17f3c1200>
<Element Proper at 0x17f3bc400>
<Element Proper at 0x17f7e6c00>
<Element Proper at 0x17f3c1200>
<Element Proper at 0x17f44a700>

INFO:	DONE
/bin/sh: nvidia-smi: command not found
/Users/evianne/Downloads/openfe-main/openfe/setup/_rbfe_utils/multistate.py:85: UserWarning: setting number of replicas to number of states: 11
  warnings.warn(msg)


Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w
        Chodera JD and Shirts MR. Replica exchange and expanded ensemble simulations as Gibbs multistate: Simple improvements for enhanced mixing. J. Chem. Phys., 135:194110, 2011. DOI:10.1063/

In [12]:
complex_settings

RelativeLigandTransformSettings(system_settings=SystemSettings(constraints='Hbonds', hydrogen_mass=None, nonbonded_method='PME', nonbonded_cutoff=<Quantity(1.0, 'nanometer')>, rigid_water=True, remove_com=True), topology_settings=TopologySettings(forcefield={'protein': 'amber99sb.xml', 'ligand': 'openff-2.0.0.xml', 'solvent': 'tip3p.xml'}, solvent_model='tip3p'), alchemical_settings=AlchemicalSettings(lambda_functions='default', lambda_windows=11, unsampled_endstates=False, use_dispersion_correction=False, softcore_LJ_v2=True, softcore_electrostatics=True, softcore_alpha=0.85, softcore_electrostatics_alpha=0.3, softcore_sigma_Q=1.0, interpolate_old_and_new_14s=False, flatten_torsions=False), integrator_settings=IntegratorSettings(timestep=<Quantity(2, 'femtosecond')>, temperature=<Quantity(298.15, 'kelvin')>, collision_rate=<Quantity(1, '1 / picosecond')>, n_steps=<Quantity(1000, 'timestep')>, reassign_velocities=True, splitting='V R O R V', n_restart_attempts=20, constraint_tolerance=